In [475]:
import pandas as pd
import seaborn as sb

#df = pd.read_csv('forestfires.csv')
df_portugal = pd.read_csv('data/Portugal_Dataset.csv')
df_portugal
#sb.heatmap(df.corr(), annot = True, fmt='.1g')


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [476]:
#Creates a dataframe containing california counties and their relative biomass(dead & alive) in tons
#4 counties unaccounted for in dead biomass so it's assumed that these areas have negligible dead biomass
df61 = pd.read_csv('data/data61.txt', delim_whitespace=True)
df61 = df61[['County', 'All_Total']]
df61 = df61.rename(columns={'All_Total': "Alive_Biomass_Tons"})
df62 = pd.read_csv('data/data62.txt', delim_whitespace=True)
df62 = df62[['County', 'All_Total']]
df62 = df62.rename(columns={'All_Total': "Dead_Biomass_Tons"})
for county in df61['County']:
    if county not in df62['County'].values:
        df62 = df62.append({'County': county,"Dead_Biomass_Tons": 0}, ignore_index=True)
df62 = df62.sort_values(by ='County', ascending = 1)
dfMass = pd.merge(df61,df62, on = ['County'])

for county in dfMass['County']:
    if "_" in county:
        fix = county.replace("_", " ")
        dfMass = dfMass.replace(county, fix)
dfMass

,County,Alive_Biomass_Tons,Dead_Biomass_Tons
0,Alameda,"5,363.1",205.4
1,Alpine,"21,755.0","3,009.8"
2,Amador,"13,987.8","1,506.8"
3,Butte,"46,750.7","3,181.1"
4,Calaveras,"27,078.1","1,732.0"
5,Colusa,"5,376.2",363.1
6,Contra Costa,"2,799.0",10.4
7,Del Norte,"81,591.3","8,065.7"
8,El Dorado,"73,636.9","7,813.7"
9,Fresno,"80,129.4","8,425.4"


In [477]:
#this function returns a dictionary containing the average weather attributes in over a fires burn time
#"start" and "end" should take the form passed from the columns: Started and Extingushed (ex. parameter: 2019-01-04T09:33:00Z)
#"stn" is the station ID
def getWeather(start, end, stn):
    last_days = {1:31, 2:28, 3:31, 4:30,5:31 ,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    year = int(start[0:4])
    month = int(start[5:7])
    day = int(start[8:10])
    if(day == 29 and month == 2):#incase of leap year
        day = 1
        month = 3
    Emonth = int(end[5:7])
    Eday = int(end[8:10])
    if(Eday == 29 and Emonth == 2):#incase of leap year
        Eday = 1
        Emonth = 3
    days = 0
    aRain = 0
    aTemp = 0
    aHum = 0 
    aWind = 0
    Eday += 1
    while(not ((day is Eday) and (month is Emonth))):
        dayWeather = df_cal_weather.loc[(df_cal_weather['Stn Id'] == stn) & (df_cal_weather['Month'] == month) & (df_cal_weather['Day'] == day) & (df_cal_weather['Year'] == year)]
        if(dayWeather.empty):
            print("No weather data for %d/%d/%d at station: %d" %(month, day, year, stn))
            return {}
        aRain += dayWeather['Precip (in)'].values
        aTemp += dayWeather['Avg Air Temp (F)'].values
        aHum  += dayWeather['Avg Rel Hum (%)'].values
        aWind += dayWeather['Avg Wind Speed (mph)'].values
        days += 1
        day += 1
        
        if(day > last_days.get(month)):
            day = 1
            month += 1
            if(month == 13):
                month = 1
                year += 1 
    aRain /= days
    aTemp /= days
    aHum /= days
    aWind /= days
    avg_weather= {"Days Burned": days,"Precipitation" : round(aRain[0], 6), "Temperature" : round(aTemp[0],6), "Humidity" : round(aHum[0],6), "Wind Speed" : round(aWind[0],6)}
    
    return avg_weather

In [478]:
# Remove irrelevant columns
# Location may be irrelevant because all are xy coordinates for Montesinho park
columns_to_keep_portugal = [
    'month', 
    'day', 
    'temp', 
    'RH',  # relative humidity
    'wind',  # wind speed
    'rain',  # rain amount
    'area'  # burned area
]

df_portugal = df_portugal[columns_to_keep_portugal]
df_portugal.shape


(517, 7)

In [479]:
# Removing all missing data
df_portugal = df_portugal.dropna()
print(df_portugal.shape)

(517, 7)


In [480]:
# Show updated dataset
df_portugal
df_portugal.to_csv('data/Portugal_Dataset_clean.csv')

In [481]:
import pandas as pd
import seaborn as sb

df_cal_weather = pd.read_csv('data/California_Weather.csv' )

#df_cal_weather

In [482]:
# Remove irrelevant columns
columns_to_keep_cal_weather = [
    'Stn Id',
    'Stn Name',
    'CIMIS Region', 
    'Date', 
    'Precip (in)',
    'Avg Air Temp (F)',
    'Avg Rel Hum (%)',
    'Avg Wind Speed (mph)',
    'Wind Run (miles)'
]

df_cal_weather = df_cal_weather[columns_to_keep_cal_weather]
df_cal_weather.shape

(128125, 9)

In [483]:
# Removing all missing data
df_cal_weather = df_cal_weather.dropna()
print(df_cal_weather.shape)

(128112, 9)


In [484]:
# Combine the data from California Weather and California Fire
# Combine based on location and Date

# Split the Date column from California Weather into month / year
df_cal_weather['Month'] = df_cal_weather['Date'].str.split('/').str[0].astype('int')
df_cal_weather['Year'] = df_cal_weather['Date'].str.split('/').str[2].astype('int')
df_cal_weather['Day'] = df_cal_weather['Date'].str.split('/').str[1].astype('int')



df_cal_weather

,Stn Id,Stn Name,CIMIS Region,Date,Precip (in),Avg Air Temp (F),Avg Rel Hum (%),Avg Wind Speed (mph),Wind Run (miles),Month,Year,Day
0,2,FivePoints,San Joaquin Valley,1/1/2018,0.00,47.8,65.0,3.3,78.3,1,2018,1
1,2,FivePoints,San Joaquin Valley,1/2/2018,0.00,47.2,67.0,3.1,74.5,1,2018,2
2,2,FivePoints,San Joaquin Valley,1/3/2018,0.00,49.9,68.0,4.5,107.5,1,2018,3
3,2,FivePoints,San Joaquin Valley,1/4/2018,0.01,56.8,74.0,5.8,140.2,1,2018,4
4,2,FivePoints,San Joaquin Valley,1/5/2018,0.00,59.8,72.0,4.2,101.4,1,2018,5
...,...,...,...,...,...,...,...,...,...,...,...,...
128120,262,Linden,San Joaquin Valley,9/14/2020,0.00,66.3,70.0,3.4,81.7,9,2020,14
128121,262,Linden,San Joaquin Valley,9/15/2020,0.00,66.0,69.0,2.7,64.2,9,2020,15
128122,262,Linden,San Joaquin Valley,9/16/2020,0.00,68.0,69.0,3.3,78.2,9,2020,16
128123,262,Linden,San Joaquin Valley,9/17/2020,0.00,70.4,65.0,3.3,78.1,9,2020,17


In [485]:
# Merge Cal Weather with CIMIS data
import glob

files_list = glob.glob("data/CIMISReports/new/*.csv")
#print(files_list)

column_headers = ['County', 'Month', 'Precip (in)', 'Avg Air Temp (F)', 'Avg Rel Hum (%)', 'Avg Wind Speed (mph)' ]

for filename in files_list:
    end_county_name = filename.index('.')
    county_name = filename[ 22:end_county_name ]
    data = {}

        
    # Initialize columns
    for header in column_headers:
        data[header] = []

    # Read each file
    df_temp = pd.read_csv(filename)
    df_cal_weather = pd.concat([df_cal_weather, df_temp])
    

In [486]:
df_cal_weather

,Stn Id,Stn Name,CIMIS Region,Date,Precip (in),Avg Air Temp (F),Avg Rel Hum (%),Avg Wind Speed (mph),Wind Run (miles),Month,Year,Day,County
0,2.0,FivePoints,San Joaquin Valley,1/1/2018,0,47.8,65,3.3,78.3,1,2018,1.0,NaN
1,2.0,FivePoints,San Joaquin Valley,1/2/2018,0,47.2,67,3.1,74.5,1,2018,2.0,NaN
2,2.0,FivePoints,San Joaquin Valley,1/3/2018,0,49.9,68,4.5,107.5,1,2018,3.0,NaN
3,2.0,FivePoints,San Joaquin Valley,1/4/2018,0.01,56.8,74,5.8,140.2,1,2018,4.0,NaN
4,2.0,FivePoints,San Joaquin Valley,1/5/2018,0,59.8,72,4.2,101.4,1,2018,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,NaN,NaN,NaN,NaN,0.25,90.4,26,51.3K,NaN,6,2020,NaN,Yolo
8,NaN,NaN,NaN,NaN,0.13K,93.8,22K,55.6L,NaN,7,2020,NaN,Yolo
9,NaN,NaN,NaN,NaN,0.00,95.7K,26K,56.1L,NaN,8,2020,NaN,Yolo
10,NaN,NaN,NaN,NaN,0.00,92.2K,24K,51.8K,NaN,9,2020,NaN,Yolo


In [487]:
#import CIMIS station data, drop Stn Name(duplicate with CA weather) and Status(irrelevant)
df_cimis = pd.read_csv('data/cimis_stations.csv', names=['Stn Id', 'Stn Name', 'Counties', 'Status'])

df_cimis = df_cimis[['Stn Id','Counties']]

#combine CIMIS data with CA weather data (CIMIS data has counties that are in the same format as in CA fire)
df_weather_with_counties = pd.merge(df_cal_weather,df_cimis, on = ['Stn Id'])



In [488]:
# Show updated dataset
"""
df_cal_weather.to_csv('data/California_Weather_clean.csv')
df_cal_weather
"""
df_weather_with_counties = df_weather_with_counties.drop(columns = ['County'])
df_weather_with_counties = df_weather_with_counties.rename(columns={"Counties": "County"})
df_weather_with_counties.to_csv('data/California_Weather_clean.csv')
df_weather_with_counties

,Stn Id,Stn Name,CIMIS Region,Date,Precip (in),Avg Air Temp (F),Avg Rel Hum (%),Avg Wind Speed (mph),Wind Run (miles),Month,Year,Day,County
0,2.0,FivePoints,San Joaquin Valley,1/1/2018,0,47.8,65,3.3,78.3,1,2018,1.0,Fresno
1,2.0,FivePoints,San Joaquin Valley,1/2/2018,0,47.2,67,3.1,74.5,1,2018,2.0,Fresno
2,2.0,FivePoints,San Joaquin Valley,1/3/2018,0,49.9,68,4.5,107.5,1,2018,3.0,Fresno
3,2.0,FivePoints,San Joaquin Valley,1/4/2018,0.01,56.8,74,5.8,140.2,1,2018,4.0,Fresno
4,2.0,FivePoints,San Joaquin Valley,1/5/2018,0,59.8,72,4.2,101.4,1,2018,5.0,Fresno
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128107,233.0,Joshua Tree,San Bernardino,9/14/2020,0,84.7,21,4.3,103.9,9,2020,14.0,San Bernardino
128108,233.0,Joshua Tree,San Bernardino,9/15/2020,0,84.1,18,3.2,75.9,9,2020,15.0,San Bernardino
128109,233.0,Joshua Tree,San Bernardino,9/16/2020,0,85.8,15,3,70.9,9,2020,16.0,San Bernardino
128110,233.0,Joshua Tree,San Bernardino,9/17/2020,0,87.1,12,4,96.5,9,2020,17.0,San Bernardino


In [489]:
print(df_weather_with_counties['County'].unique())
print(df_weather_with_counties.columns)

['Fresno' 'Kern' 'Yolo' 'Butte' 'El Dorado' 'Kings' 'Inyo' 'Imperial'
 'Shasta' 'Riverside' 'Contra Costa' 'San Luis Obispo' 'Merced' 'Lassen'
 'Santa Barbara' 'San Joaquin' 'Stanislaus' 'Orange' 'Napa' 'Los Angeles'
 'Sonoma' 'Yuba' 'Tulare' 'Modoc' 'Siskiyou' 'Santa Cruz' 'Mendocino'
 'Monterey' 'San Bernardino' 'Solano' 'San Benito' 'Sacramento'
 'San Diego' 'Ventura' 'Marin' 'Alameda' 'Placer' 'Santa Clara' 'Tehama'
 'Amador' 'Sutter' 'Colusa' 'San Mateo' 'Alpine' 'Humboldt']
Index(['Stn Id', 'Stn Name', 'CIMIS Region', 'Date', 'Precip (in)',
       'Avg Air Temp (F)', 'Avg Rel Hum (%)', 'Avg Wind Speed (mph)',
       'Wind Run (miles)', 'Month', 'Year', 'Day', 'County'],
      dtype='object')


In [490]:
import pandas as pd
import seaborn as sb

df_cal_fire = pd.read_csv('data/California_Fire_Incidents.csv')
#df_cal_fire


In [491]:
# Remove irrelevant columns
columns_to_keep_cal_fire = [
    'AcresBurned',
    'ArchiveYear',
    'CalFireIncident',
    'Counties', 
    'Extinguished',
    'Location',  
    'Started'
]

df_cal_fire = df_cal_fire[columns_to_keep_cal_fire]
df_cal_fire.shape

(1636, 7)

In [492]:
# Removing all missing data
df_cal_fire = df_cal_fire.dropna()
print(df_cal_fire.shape)

(1577, 7)


In [493]:
# Show updated dataset
df_cal_fire
df_cal_fire.to_csv('data/California_Fire_Incidents_clean.csv')

In [494]:
# Split the Started column from California Weather into month / year
df_cal_fire['year'] = df_cal_fire['Started'].str.split('-').str[0].astype('int')
df_cal_fire['month'] = df_cal_fire['Started'].str.split('-').str[1].astype('int')

# Rename month/year columns
df_cal_fire = df_cal_fire.rename(columns={"Counties": "County"})
df_cal_fire = df_cal_fire.rename(columns={"month": "Month", "year": "Year"})

df_cal_fire


,AcresBurned,ArchiveYear,CalFireIncident,County,Extinguished,Location,Started,Year,Month
0,257314.0,2013,True,Tuolumne,2013-09-06T18:30:00Z,3 miles east of Groveland along Hwy 120,2013-08-17T15:25:00Z,2013,8
1,30274.0,2013,True,Los Angeles,2013-06-08T18:30:00Z,Angeles National Forest,2013-05-30T15:28:00Z,2013,5
2,27531.0,2013,True,Riverside,2013-07-30T18:00:00Z,Hwy 243 & Hwy 74 near Mountain Center,2013-07-15T13:43:00Z,2013,7
3,27440.0,2013,False,Placer,2013-08-30T08:00:00Z,"Deadwood Ridge, northeast of Foresthill",2013-08-10T16:30:00Z,2013,8
4,24251.0,2013,True,Ventura,2013-05-11T06:30:00Z,Southbound Highway 101 at Camarillo Springs Ro...,2013-05-02T07:01:00Z,2013,5
...,...,...,...,...,...,...,...,...,...
1627,10.0,2019,True,Riverside,2019-09-10T10:45:00Z,Bailey Road and Terwilliger Road in Anza,2019-09-10T10:43:58Z,2019,9
1629,10.0,2019,True,Siskiyou,2019-06-21T14:23:00Z,"Deer Run Road, east of Iron Gate Reservoir",2019-06-16T20:33:00Z,2019,6
1630,10.0,2019,True,Butte,2019-05-08T08:47:00Z,"Off Hamilton Nord Cana Hwy & Cana Hwy, northea...",2019-04-30T12:20:00Z,2019,4
1631,9.0,2019,True,Riverside,2019-10-10T18:11:00Z,"Eagle Canyon Rd. and Cajalco Rd., southwest of...",2019-10-10T12:08:00Z,2019,10


In [495]:
# FIXME

#combine matching entries of CA fire and CA weather by County, month, and year
df_CA_merged = pd.merge(df_cal_fire,df_weather_with_counties, how = 'inner', on = ['County','Month','Year'])

df_CA_merged


,AcresBurned,ArchiveYear,CalFireIncident,County,Extinguished,Location,Started,Year,Month,Stn Id,Stn Name,CIMIS Region,Date,Precip (in),Avg Air Temp (F),Avg Rel Hum (%),Avg Wind Speed (mph),Wind Run (miles),Day
0,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250.0,Williams,Sacramento Valley,7/1/2018,0,82.7,52,4.8,114.4,1.0
1,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250.0,Williams,Sacramento Valley,7/2/2018,0,75.8,67,5,119.4,2.0
2,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250.0,Williams,Sacramento Valley,7/3/2018,0,73.9,63,5,119.9,3.0
3,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250.0,Williams,Sacramento Valley,7/4/2018,0,67.6,72,5.6,135.6,4.0
4,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250.0,Williams,Sacramento Valley,7/5/2018,0,71,65,3.8,90.6,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56323,10.0,2019,True,Butte,2019-05-08T08:47:00Z,"Off Hamilton Nord Cana Hwy & Cana Hwy, northea...",2019-04-30T12:20:00Z,2019,4,244.0,Biggs,Sacramento Valley,4/26/2019,0,72.5,48,3.1,73.8,26.0
56324,10.0,2019,True,Butte,2019-05-08T08:47:00Z,"Off Hamilton Nord Cana Hwy & Cana Hwy, northea...",2019-04-30T12:20:00Z,2019,4,244.0,Biggs,Sacramento Valley,4/27/2019,0,70.3,57,4.6,111.2,27.0
56325,10.0,2019,True,Butte,2019-05-08T08:47:00Z,"Off Hamilton Nord Cana Hwy & Cana Hwy, northea...",2019-04-30T12:20:00Z,2019,4,244.0,Biggs,Sacramento Valley,4/28/2019,0,66,61,3.1,74.2,28.0
56326,10.0,2019,True,Butte,2019-05-08T08:47:00Z,"Off Hamilton Nord Cana Hwy & Cana Hwy, northea...",2019-04-30T12:20:00Z,2019,4,244.0,Biggs,Sacramento Valley,4/29/2019,0,66.3,61,5,119.3,29.0


In [496]:
# Show updated dataset
df_CA_merged
df_CA_merged.to_csv('data/California_Merged_clean.csv')

In [497]:
getWeather('2018-07-27T12:05:00Z','2019-01-04T09:33:00Z', 250)

{'Days Burned': 162,
 'Precipitation': 0.019444,
 'Temperature': 60.873457,
 'Humidity': 64.654321,
 'Wind Speed': 4.47284}

In [498]:
"""
# Get the unique Regions/Counties of weather/fire and use 'str.replace()' to merge identical values

# Cleaning up Cal_Fire Counties to match Cal_weather Regions Choices

# San Francisco Bay
sf_bay = ['Alameda', 'Amador', 'Marin', 'Napa', 'San Mateo', 'Santa Clara', 'Sonoma']
# ?
cent_cal = ['Fresno', 'Monterey','San Joaquin Valley']
# Sacramento Valley
sac_valley = ['Butte', 'Sacramento Valley', 'Sutter', 'Yuba']
# North Coast Valleys
north_coast = ['Humboldt','Mendocino']
# ?
in_so_cal = ['Kern', 'San Bernardino']
# Los Angeles Basin
la_basin = ['Los Angeles']
# ?
great_basin = ['Modoc']
# Sierra Foothill
sierra_nv = ['Nevada', 'Sierra', 'State of Nevada']
# South Coast Valleys
so_coast = ['Orange', 'San Diego']
# Central Coast Valleys
cent_coast = ['San Luis Obispo', 'Santa Barbara', 'Santa Cruz']
# ?
shasta_cas = [ 'Shasta', 'Trinity']

# Unknown:
# Alpine, Calaveras, Colusa, Contra Costa, Del Norte, El Dorado, Glenn, Inyo, Kings, Lake, Lassen, Madera
# Mariposa, Merced, Mexico, Mono, Placer, Plumas, Riverside, San Benito, Siskiyou, Solano, Stanislaus
# State of Oregon, Tehama, Tulare, Tuolumne, Ventura, Yolo,


df_cal_fire['Counties'] = df_cal_fire['Counties'].str.replace('Alameda', 'San Francisco Bay')

regions_list = {'SanFranciscoBay': sf_bay, 'CentralCal': cent_cal, 'SacramentoValley': sac_valley, 
                'NorthCoastValleys': north_coast, 'InlandSoCal': in_so_cal, 'LosAngelesBasin': la_basin, 
                'GreatBasin': great_basin, 'SierraFoothill': sierra_nv, 'SouthCoastValleys': so_coast, 
                'CentralCoastValleys': cent_coast, 'ShastaCascade': shasta_cas}

for region_name in regions_list:
    for county in regions_list[region_name]:
        # FIXME
        df_cal_fire['Counties'] = df_cal_fire['Counties'].str.replace(county, region_name)

unique_regions = (df_cal_weather['CIMIS Region'].unique())
print(sorted(unique_regions))
unique_counties = (df_cal_fire['Counties'].unique())
print(sorted(unique_counties))

"""

"\n# Get the unique Regions/Counties of weather/fire and use 'str.replace()' to merge identical values\n\n# Cleaning up Cal_Fire Counties to match Cal_weather Regions Choices\n\n# San Francisco Bay\nsf_bay = ['Alameda', 'Amador', 'Marin', 'Napa', 'San Mateo', 'Santa Clara', 'Sonoma']\n# ?\ncent_cal = ['Fresno', 'Monterey','San Joaquin Valley']\n# Sacramento Valley\nsac_valley = ['Butte', 'Sacramento Valley', 'Sutter', 'Yuba']\n# North Coast Valleys\nnorth_coast = ['Humboldt','Mendocino']\n# ?\nin_so_cal = ['Kern', 'San Bernardino']\n# Los Angeles Basin\nla_basin = ['Los Angeles']\n# ?\ngreat_basin = ['Modoc']\n# Sierra Foothill\nsierra_nv = ['Nevada', 'Sierra', 'State of Nevada']\n# South Coast Valleys\nso_coast = ['Orange', 'San Diego']\n# Central Coast Valleys\ncent_coast = ['San Luis Obispo', 'Santa Barbara', 'Santa Cruz']\n# ?\nshasta_cas = [ 'Shasta', 'Trinity']\n\n# Unknown:\n# Alpine, Calaveras, Colusa, Contra Costa, Del Norte, El Dorado, Glenn, Inyo, Kings, Lake, Lassen, Madera